In [8]:
%load_ext autoreload
%autoreload 2
%conda install "networkx<2.7"  
#for some reason scipy>1.8 won't install for me, this is to make sparse matrices work

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
zsh:1: no such file or directory: 2.7

Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
import scipy
import pandas as pd
import os
import os.path
import matplotlib.pyplot as plt
import numpy as np
import random
import networkx as nx
from pathlib import Path

In [2]:
print (nx.__version__)
print(scipy.__version__)

2.8.8
1.7.3


In [ ]:
#optional cell if using drive:
#if using drive the file creeation will not work!
#from google.colab import drive
#drive.mount('/content/gdrive')



# Section 0: Getting data from million songs playlists

### How to use this notebook

1. Download it
2. Download and extract the million songs dataset (link in the README.md on the repo) in the same directory as this notebook
3. Set the DIRECTORY constant in the cell below as the directory containing the dataset
4. You should then be able to run all of it

Section 1 is making a graph to figure out how the number of followers over all the playlists is distributed
Section 2 is removing all playlists from the dataset which have N or more fewer, and creating a new file containing playlists with only more than N followers
Section 3 is extracting N songs randomly from a given dataset, either randomly or with a weighted random distribution. You can set the constants to change the sampling method, number of songs, and the dataset you are computing over. If you use the default dataset to pick the songs, it is quite slow (takes like 10 minutes overall), so you may want to use a smaller dataset to choose these songs. I set the dataset to the dataset containing playlists with over 5 followers and it is significantly quicker to run. 

### IMPORTANT: Set the directory containing the data below

In [2]:
DIRECTORY = "spotify_million_playlist_dataset/data/"

### 1. Figuring out distribution of number of followers over playlists

This is just to figure out what the distribution of the followers looks like

In [ ]:
# 1. Followers numbers
# 10,000 songs choose (Quanchi's link)


num_followers = []
directory = os.listdir(DIRECTORY)

for file in directory:

    f = open(DIRECTORY+file)
    data = json.load(f)


    for playlist in range(len(data['playlists'])):
        x = data['playlists'][playlist]['num_followers']
        num_followers += [x]

num_followers = pd.Series(num_followers)
num_followers.value_counts()


In [ ]:
num_likes = {}

for val in num_followers:
    if val in num_likes:
        num_likes[val] += 1
    else:
        num_likes[val] = 1

keys = []
for key in num_likes.keys():
    keys += [key]

keys.sort()

X,y = [],[]

for key in keys:
    X +=[key]
    y +=[num_likes[key]]
    
    
#make the y-values a fraction of the size of the dataset
y_c = y     
for i in range(len(y_c)):
        y_c[i] = (y_c[i]+y_c[i-1])

for i in range(len(y_c)):
    y_c[i]/= 1000000

y_c


    
    

In [ ]:
#Plot

plt.plot(X[:40], y[:40],linestyle="-", marker="o")
plt.ylabel("Fraction of playlists with followers <= than this number")
plt.xlabel("Number of Followers")
plt.xticks(np.arange(0,41,2))
plt.show()
f = plt.figure()
f.set_figwidth(1)
f.set_figheight(1)

In [ ]:
print(1000000 - y_c[5]*1000000)


### 2. Removing the majority of the playlists

Removing all playlists with N or less followers 

In [ ]:
N = 10
directory = os.listdir(DIRECTORY)

moreThanN = []

for file in directory:

    f = open(DIRECTORY+file)
    data = json.load(f)


    for playlist in range(len(data['playlists'])):
        if data['playlists'][playlist]['num_followers'] >N:  
            moreThanN += [data['playlists'][playlist]]

In [ ]:
#sanity check

len(moreThanN)

In [ ]:
#Create results folder

path = os.getcwd()+"/generated_data"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(path)
   print("The new directory is created!")

In [ ]:
#SAVING DATA
#convert to dictionary 
moreThanN_dict = {"playlists": moreThanN}

json_object = json.dumps(moreThanN_dict, indent=4)
with open(path+"/moreThan"+str(N)+"followers.json", "w") as outfile:
    outfile.write(json_object)



In [ ]:
#formatting stuff for .edges filef

data = ""

for key in edges:
    data+= (key + ' ' + str(edges[key])+'\n')

In [ ]:
#writing the truncated data to a new .json file

with open(path+"/moreThan"+str(N)+"followers.edges", "w") as outfile:
    outfile.write(data)

### 3. Choosing our songs and generating data

Selecting N songs randomly and performing an analysis on them.

Change the constants in the cell below to change the kind of data you create.

The output of these cells will be a .csv file in the generated_data folder.

In [3]:
################# Constants #################
'''Rename this constant to the file containing the data, otherwise it will go through all of the data.
I recommend using a file generated by Method 1 as the specific file (for instance, moreThan10followers.json) since
the computation times will be much much faster'''
SPECIFIC_FILE = 'generated_data/moreThan10followers.json' 

WEIGHTED_RANDOM = False   #If we want weighted random sampling or not
N = 2000  #number of songs to consider

############################################

In [4]:

#first we need to load all the songs into a dictionary so that we can randomly select N of them


all_tracks = {} #stores all the songs
count = 0 #progress bar


if SPECIFIC_FILE == '':
    directory = os.listdir(DIRECTORY)
    for file in directory:
        count +=1
        if (count %100 == 0):
            print(count)
        f = open(DIRECTORY+file)
        data = json.load(f)

        for playlist in data['playlists']:
            for track in playlist['tracks']:
                if track['track_name'] not in all_tracks:  #if it not in the dict then we add it to it and increase its weight
                    all_tracks[track['track_name']] = 1 #weight is used for weighted random sampling (if needed)
                else:
                    all_tracks[track['track_name']] +=1
                
############### TO LOOP OVER SOME SMALLER FILE ##################\
else:
    f = open(SPECIFIC_FILE)
    data = json.load(f)

    for playlist in data['playlists']:
        for track in playlist['tracks']:
            if track['track_name'] not in all_tracks:  #if it not in the dict then we add it to it and increase its weight
                all_tracks[track['track_name']] = 1 #weight is used for weighted random sampling (if needed)
            else:
                all_tracks[track['track_name']] +=1



In [5]:
print(list(all_tracks.keys())[0:20])

['T-Shirt', 'It All Started with a Beer', "Night's On Fire", 'From the Ground Up', 'Crash And Burn', 'Break Up with Him', "I Don't Dance", 'Dirt', 'Die A Happy Man', 'It Goes Like This', "Don't It", '19 You + Me', "Nothin' Like You", 'I Know Somebody', "It Don't Hurt Like It Used To", 'Vacation', 'Different For Girls', 'Sleep Without You', 'Star Of The Show', 'Sunshine & Whiskey']


In [6]:
#get N songs based on weighted random sampling (no particular reason for this apart from probably giving a more connected NW)

if WEIGHTED_RANDOM == False:
    songs = random.choices(list(all_tracks.keys()), weights=None, k=N)
else:
    songs = random.choices(list(all_tracks.keys()), weights=all_tracks.values(), k=N)


songs = set(songs) #convert to set
print(songs)

{'Cat Skillz', 'No Me Quites Tu Amor', 'Cuando Baila Reggaeton', 'Baby Girl', 'Kya Karoon? (From "Wake Up Sid ")', 'Let Down (feat. Andrew Harris)', 'A Quiet Place', 'ไม่มีใครยอมใคร (เพลงประกอบละคร ชิงรัก ริษยา)', 'Old Fashioned', 'Para Volver a Comenzar', 'Ella me Da', '僕の手に触れるな', 'In The Stories', 'Extremely Loud and Incredibly Close (From "Extremely Loud and Incredibly Close")', 'Better Man - Remastered', 'Everyone Deserves Music', 'No Far Away', '我愛台妹', "You'll Accomp'ny Me", "Say You'll Be There - Single Mix", 'Bury Our Love', 'Duduk Solo', 'Amanecí en Tus Brazos', 'The Beauty Within', 'Ideekay', 'That\'s How You Know - From "Enchanted"/Soundtrack Version', 'Makina', 'Shake It Up - Live', 'Little White Lie', 'I Believe In Symmetry', 'Tickle', 'Save As Draft', 'Juegos De Seducción - Remasterizado 2007', 'Polka Dots And Moonbeams', 'Prove Me Right', 'Just Enough (feat. Charlene Marie)', 'Él No Te Da - Club Mix', 'Oscar Barraza', 'Rent I Pay', "Now That She's Gone", '537 C U B A', 'L

In [7]:
#Now I want to create a list of shared playlists

shared_playlists = [] #this will contain items of the form (weight: [weight], tracks: [tracks]) for each playlist

if SPECIFIC_FILE == '':
    for file in directory:
        f = open(DIRECTORY+file) 
        data = json.load(f)

        for playlist in data['playlists']:  #parse through every playlist

            num_followers = playlist['num_followers']

            temp = []

            for track in playlist['tracks']:
                if track['track_name'] in songs and track['track_name'] not in temp:
                    temp.append(track['track_name'])

            if (len(temp) >1): #only if a playlist has two or more of the 10k songs then we add it to our shared_playlists lits
                item = {'weight':num_followers, 'tracks':temp}
                shared_playlists.append(item)

else:
    
    f = open(SPECIFIC_FILE) 
    data = json.load(f)

    for playlist in data['playlists']:  #parse through every playlist

        num_followers = playlist['num_followers']

        temp = []

        for track in playlist['tracks']:
            if track['track_name'] in songs and track['track_name'] not in temp:
                temp.append(track['track_name'] + ", by "+ track['artist_name'])

        if (len(temp) >1): #only if a playlist has two or more of the 10k songs then we add it to our shared_playlists lits
            item = {'weight':num_followers, 'tracks':temp}
            shared_playlists.append(item)

            
                

In [8]:
#create edges
edges = {}
count = 0
for item in shared_playlists:

    weight = item['weight']
    tracks = item['tracks']
    for track1 in tracks:
        for track2 in tracks:
            if (track1 == track2):
                continue
            if (track1,track2) in edges:
                edges[(track1,track2)] += weight
            elif ((track2,track1)) in edges:
                edges[(track2,track1)] += weight
            else:
                edges[(track1,track2)] = weight
            

In [9]:
print(list(edges.items())[0:5])

[(('Dirt, by Florida Georgia Line', 'Middle of a Memory, by Cole Swindell'), 1034), (('Dirt, by Florida Georgia Line', "Lovin' Lately, by Big & Rich"), 910), (('Middle of a Memory, by Cole Swindell', "Lovin' Lately, by Big & Rich"), 1236), (("Suite d'Arrée: Bennilis - Nestaval - Trédudon + Trévezel - Arrée, by Gwendal", 'A Tune For Frankie, by Altan'), 28), (('Better Man - Remastered, by Pearl Jam', 'Good Feeling, by Flo Rida'), 88)]


In [10]:
data = []

for key in edges:
    temp = (key[0],key[1],edges[key]) #make tuple of form (song1, song2, weight)
    data.append(temp)
data2 = pd.DataFrame(data)
data2.columns = ["Source","Target","Weight"]
print(data2[0:5]) #sanity check

                                              Source  \
0                      Dirt, by Florida Georgia Line   
1                      Dirt, by Florida Georgia Line   
2               Middle of a Memory, by Cole Swindell   
3  Suite d'Arrée: Bennilis - Nestaval - Trédudon ...   
4              Better Man - Remastered, by Pearl Jam   

                                 Target  Weight  
0  Middle of a Memory, by Cole Swindell    1034  
1          Lovin' Lately, by Big & Rich     910  
2          Lovin' Lately, by Big & Rich    1236  
3          A Tune For Frankie, by Altan      28  
4             Good Feeling, by Flo Rida      88  


In [11]:
#Create results folder

path = os.getcwd()+"/generated_data"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(path)
   print("The new directory is created!")

In [12]:
#Writing the file

name = ""

if (WEIGHTED_RANDOM == True):
    name = str(N) + '_weighted'
else:
    name = str(N)
filename = path+'/'+name+"_random_songs.csv"
filepath = Path(filename)
while (os.path.exists(filepath)):
    if filename[-5]=='s':
        filename = filename[:-4]+'2'+filename[-4:]
    else:
        filename = filename[:-5]+str(int(filename[-5])+1)+filename[-4:]
    filepath = Path(filename)

data2.to_csv(filepath, index = False)
    
    
#The data should be saved in the same folder as this notebook

## Thoughts so far

- Selecting playlists above a certain number of likes produces a very dense network even with only the top 600~ networks
- Selecting 10k songs randomly produces a pretty sparse network with some intereseting communities
- Selecting 10k songs randomly with a weighted distribution produces an extremely dense network
- The clustering coefficient for 100 random weighted songs is small (0.04), but when we pick 500 random weighted songs it grows larger (0.34). For 10000 random songs it was like 0.99

## 0.2 Implementing Networkx

The purpose of this subsection is to use networkx to load our .csv files that we have produced, so that we can create adjacency matrices, figure out shortest path lengths, etc



In [27]:
path = 'generated_data/2000_random_songs.csv' #file to be read

In [28]:
df = pd.read_csv(path)
G = nx.from_pandas_edgelist(df, "Source", "Target", edge_attr = "Weight")  #generate graph
print(nx.adjacency_matrix(G).todense())

/var/folders/xr/swmcxvvx1v97vbgsfrw5dbyw0000gn/T/ipykernel_3943/382350200.py:3: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  print(nx.adjacency_matrix(G).todense())


AttributeError: module 'scipy.sparse' has no attribute 'coo_array'

# Section 1: Defining the edges of the network

In this section we explore several different means to quantify how similar one song is to another. We will use these similarity scores to assign weights to the edges of our network. 

### Idea 1: Shared Followers
This method assigns a weight to each edge based on the sum of likes of each playlist that both songs in the edge appear in.

$w_{i,j} = \Sigma_{p \in P_{i,j}}{\normalsize \text{# likes of }p}$, where $P_{i,j}$ is the set of playlists contatining songs $i$ and $j$

### Idea 2: Shared Playlists
The weight of each edge is the number of shared playlists of the song

$w_{i,j} = \normalsize |P_{i,j}|$

### Idea 3: Hybrid of 1 and 2
Since the follower numbers of the playlists are heavily skewed, ranging from 1 to over 70k, we experiment with weighting the shared playlist number higher than the shared follower number.

$w_{i,j} = \normalsize |P_{i,j}| + log_2({\Sigma_{p \in P_{i,j}}{\text{# likes of }p}})$

For our purposes, we use $log_2$ to weight the follower numbers as we think it suits our data well.

### Idea 4: Shared Playlist Simliarity

Let $p_{i,j}$ denote the fraction of playlists that song $i$ belongs to which also contain song $j$. 
Then, $ w_{i,j} = \large \frac{p_{i,j}\cdot p_{j,i}}{2}$

I.e, this is the average of the fraction of all playlists containing one song that also contain the other


## TODO: Define functions that implement each of these ideas

Each function should take in a set of N songs, an input file (not the whole dataset as that is way too big!) and return two things: 1. The edges of the graph created by each method, and 2. A dictionary of node relations to each other.


So what we're doing here is 
- selecting N songs (like 10k out of the million) out of the input file (has more than N songs).  
- We want to remember only the playlists with two or more of the N songs in them. For that playlist, we remember it as a pair of (weight, [tracks out of the N in that playlist]) (but you could play around with this to see what works) 
- Then, using the cleaned up, relevant playlists with their weights (that we did in the previous step) we make the graph, which is a dictionary of the form { {(node1,node2), weight} }. So, the key is (node1,node2) and the corresponding value is weight

Ultimately the output should be the network with the weights aligned with each of the above ideas. 

# Section 2: Implementing algorithm to generate playlists

## 2.1: Star Graph Algorithm

Here I'll try to create the playlists using the star graph recommendation method. For this I need two things - first is a recommendation algorithm based on the edges, and second is an easy way to iterate over song connections. 

### How this works:

First we consider the one node case. We feed a single node, call it node A, into the algorithm which gives us the subset of the network with one edge at A, which is a star graph. We store this star graph in a dictionary object. The next song to be recommended is one of the edges in this graph, randomly chosen with a weighted probability distribution. Call this song B. Now, we want the playlist that we are generating to be the center of the star graph. Now, we make the center of the star graph represent both nodes A and B. We add all the edges with end to B (except those with end A) from the original dataset to the star graph. If an edge already exists between the center and another node, and B is also connected with that node, the new weight of that edge becomes the sum of the weights of those edges. This is to implement familiarity. Continue this until the desired playlist size is reached. 


### TODO:
- If we can create functions for creating the edge weights of the network that would be totally awesome
- Perhaps cosine simliarity would be a good thing to explore for our recommendations?

Running the two cells below will initialize the network. There are some constants at the start that you need to input

In [ ]:
######################################### Constants

SPECIFIC_FILE = 'generated_data/moreThan10followers.json' #Rename this to something else if you want to only consider some other dataset, 
                    #otherwise we go over all data. specify the file name/path as the name
WEIGHTED_RANDOM = False   #If we want weighted random sampling or not
N = 1000  #number of songs to consider

In [ ]:
######################################### LOADING SONGS FROM FILE
#first we need to load all the songs into a dictionary so that we can randomly select N of them


all_tracks = {} #stores all the songs
count = 0 #progress bar


f = open(SPECIFIC_FILE)
data = json.load(f)

for playlist in data['playlists']:
    for track in playlist['tracks']:
        if track['track_name'] not in all_tracks:  #if it not in the dict then we add it to it and increase its weight
            all_tracks[track['track_name']] = 1 #weight is used for weighted random sampling (if needed)
        else:
            all_tracks[track['track_name']] +=1


######################################### CHOOSING RANDOM SONGS

if WEIGHTED_RANDOM == False:
    songs = random.choices(list(all_tracks.keys()), weights=None, k=N)
else:
    songs = random.choices(list(all_tracks.keys()), weights=all_tracks.values(), k=N)

#print(songs)

######################################### CREATING SHARED PLAYLIST EDGES

shared_playlists = [] #this will contain items of the form (weight: [weight], tracks: [tracks]) for each playlist


f = open(SPECIFIC_FILE) 
data = json.load(f)

for playlist in data['playlists']:  #parse through every playlist

    num_followers = playlist['num_followers']
    temp = []
    for track in playlist['tracks']: #if one of the random songs is in the playlist then remember it
        if track['track_name'] in songs and track['track_name'] not in temp:
            temp.append(track['track_name'])

    if (len(temp) >1): #only if a playlist has two or more of the 10k songs then we add it to our shared_playlists lits
        item = {'weight':num_followers, 'tracks':temp}
        shared_playlists.append(item)


# #Here we create the big graph
edges = {}
relations = {} #this variable stores all of the node pairs in the graph. It is useful for the playlist-generation part of the notebook
for song in songs:
    relations[song] = set({})

for item in shared_playlists:
    
    weight = item['weight']
    tracks = item['tracks']
    for track1 in tracks:
        for track2 in tracks:
            if (track1 == track2):
                continue
            #now we add relations
            if track2 not in relations[track1]:
                relations[track1].add(track2)
            if track1 not in relations[track2]:
                relations[track2].add(track1)
                
            #now create edge and weight
            if (track1,track2) not in edges and (track2,track1) not in edges:
                edges[(track1,track2)] = weight
                edges[(track2,track1)] = weight
            else:
                edges[(track1,track2)] += weight
                edges[(track2,track1)] += weight


print("Some songs that were selected:")
print(songs[0:5])
print('\n')
print("Some of the edges that were created:")
print(list(edges.items())[0:5])
print('\n')
print("Some of the pairs of nodes in the graph:\n")
for n in range(5):
    i = random.randint(0,len(songs))
    print("Track:"+str(songs[i]))
    print("Tracks it is related to:")
    print(relations[songs[i]])
    print('\n')





The cell below defines the function that we use

In [ ]:
def recommend_song(stargraph, songs, alpha = 1):
    '''
    Function that will implement the recommend algorithm for a given input star graph. We are assigning weights to every 
    song in the number of songs we are operating over. The weight of each song will be the weight of the edge from it to the
    center of the stargraph (0 otherwise) + alpha. We will then make a weighted random prediction to get the next song.
    
    Parameters:
        stargraph: the input star graph. The first element is the nodes inside the graph and the second one is a dictionary 
        of node:weight pairs
        songslist: the list of songs over which we are operating
        alpha: value to give to every song to allow possible recommendation of songs without any edges (teleportation). 1 by default
  
  Returns:
       The predicted song
   '''
    weights = []
    for i in range(len(songs)):
        song = songs[i]
        weight = alpha
        if song in stargraph[1]: #if there is already an edge to it
            weight+= stargraph[1][song]
        if song in stargraph[0]: #if it is already in our playlist
            weight = 0
        weights.append(weight)
    nextsong = random.choices(songs, weights, k=1)[0] #return our weighted random choice
    #print (nextsong)
    return nextsong

def update_graph(stargraph, songToAdd, relations, edges):
    '''
    Function that will update the star graph to include edges that belong to the song to add. Returns updated star graph
    
    Parameters:
        stargraph: the input star graph. The first element is the nodes inside the graph and the second one is a dictionary 
        of node:weight pairs
        songToAdd: the song to add
        relations: the dictionary of all pairs in the graph
        edges: the original network with all the node pair relations
   
   Returns:
        The updated star graph
    '''
    relationsToAdd = relations[songToAdd] #give a set of all the pairs of nodes with the song to add
    stargraphNodes = stargraph[0]
    stargraphEdges = stargraph[1]
    
    for song in relationsToAdd: #go through each pair belonging to this song
        
        if song not in stargraphNodes:
            if song not in stargraphEdges:
                stargraphEdges[song] = edges[(song,songToAdd)] #add value of edge from the original network
            else:
                stargraphEdges[song] += edges[(song,songToAdd)]
                
    stargraph[0].add(songToAdd)
    return stargraph
    
    
def create_playlist(inputSongs, length,  songs, relations, edges,alpha = 1):
    '''
    Function that generates a recommendation playlist of length length based on an input of songs, and an input network
    
    Parameters:
    inputSongs: The input songs (list)
    length: Number of songs to add
    songs: List of all songs
    relations: Dictionary of all pair relations in the graph
    edges: The original network
    alpha: The teleportation probability in the recommendation algorithm
    '''
    stargraph = (set(), dict())
    
    for song in inputSongs: #build the star graph
        stargraph = update_graph(stargraph, song, relations, edges) 
    
    for i in range(length):
        newSong = recommend_song(stargraph, songs, alpha)
        stargraph = update_graph(stargraph, newSong, relations, edges)
    
    return list(stargraph[0])
    
    
            
        
        
    
    

## Using the algorithm

Finally, the cell below tests the algorithm

In [ ]:
#Testing the algorithm

playlist = []
for i in range(3):
    playlist.append(songs[random.randint(0,len(songs))])
length = len(playlist)+10
print("Input songs:")
print(playlist)
playlist = create_playlist(playlist, length, songs, relations, edges)
print("Generated playlist:")
print(playlist)


### Thoughts on algorithm

I have no idea if these are great playlists or not lmao

## 2.2 Cosine Simliarity

### TODO
- maybe this would be good for idea #4 of Section 1 since the weights are all normalized and in the [0,1] range